<a href="https://colab.research.google.com/github/vishKurama/financialDividends/blob/master/IRScraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!pip install google_images_download

     |████████████████████████████████| 911kB 3.7MB/s 
  Created wheel for google-images-download: filename=google_images_download-2.8.0-py2.py3-none-any.whl size=14550 sha256=a5e6f4b702656bad9cfde3638e33c669d5f8369b89b20ca2725ed82f665732aa
  Stored in directory: /root/.cache/pip/wheels/1f/28/ad/f56e7061e1d2a9a1affe2f9c649c2570cb9198dd24ede0bbab
Successfully built google-images-download


In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
# wd.get("https://www.webite-url.com")

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
chr

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: use options instead of chrome_options
  app.launch_new_instance()


In [ ]:
!ls

sample_data


In [6]:
# The goal is to reach the news announcements
from bs4 import BeautifulSoup
import requests
import csv
import re
from urllib.parse import urljoin, urlparse
import time
# chromeOptions = webdriver.ChromeOptions()
# chromeOptions.add_argument("--headless")
# driver = webdriver.Chrome(options=chromeOptions)
urls = []
visited_urls = {}
dividend_keywords = ['Dividend', 'Dividends']
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'}
iterator = 0
previous_base_uri = '$'
investor_relations_keywords = ['Investor Relations', 'Investor', 'Corporate Actions', 'Investors']
press_release_keywords = ['Press Release', 'News Announcement', 'News Announcements', 'Press Releases', 'News Releases', 'news room', 'News Room']
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36', }

urls = ['https://www.internationalpaper.com/performance/investors', 'http://www.hyster-yale.com/Investors/default.aspx', 'http://www.1stconstitution.com/investorRelations/', 'http://www.ahbelo.com/']

def get_domain_name(url):
  try:
    domain = urlparse(url).netloc
    domain = domain.split('.')[1]
  except:
    domain = "default"
  return domain

def save_news_announcements(base, url):
  global iterator
  try:
      driver.get(url)
      page = driver.page_source
      f = open('news/' + base + str(iterator) + '.html', 'w')
      f.write(page)
      iterator += 1
  except Exception as err:
    print (err)
      

def get_news_announcements(base, urls):
  for url in urls:
    print (url)
    try:
      driver.get(url)
      page = driver.page_source

    except Exception as err:
      print (err)
      continue
    soup = BeautifulSoup(page, 'lxml')
    links = []
    for link in soup.findAll('a'):
      if "dividend" in link.text.lower():
        save_news_announcements(base, urljoin(url, link.get('href')))


def find_news_announcements():
  with open('Company URL - Sheet1.csv') as f:
        reader = csv.DictReader(f)
        for row in reader:
            if row['COUNTRY'] == 'USA':
                urls.append(row['URL'])
  


  for url in urls:
    base = get_domain_name(url)
    global previous_base_uri, iterator
    previous_base_uri = base
    iterator = 0
    print (base)
    try:
      driver.get(url)
      page = driver.page_source
    except Exception as err:
      print (err)
      continue
    soup = BeautifulSoup(page, 'lxml')
    links = []
    for link in soup.findAll('a'):
      if link.text in press_release_keywords:
        links.append(urljoin(url, link.get('href')))

    get_news_announcements(base, links)


def get_dividends(possible_urls):
    for url in possible_urls:
        if url in visited_urls.keys():
            continue
        print('ON url, find dividend :: ', url)
        try:
            # page = requests.get(url, headers=headers, timeout=20).text
            driver.get(url)
            page = driver.page_source

            if "Page Not Found" in page or "Bad Request" in page or "System Error" in page or "System error" in page:
                continue
        except Exception as err:
            print(err)
            continue
        soup = BeautifulSoup(page, 'lxml')
        visited_urls[url] = True
        global iterator
        f = open('html/dividend_' + str(iterator) + '.html', 'w')
        f.write(page)
        iterator += 1


def go_to_ir(possible_urls):
    for url in possible_urls:
        if url in visited_urls.keys():
            continue
        print('ON url, go to ir :: ', url)
        try:
            # page = requests.get(url, headers=headers, timeout=20).text
            driver.get(url)
            page = driver.page_source
        except Exception as err:
            print(err)
            continue
        visited_urls[url] = True
        soup = BeautifulSoup(page, 'lxml')
        for a in soup.find_all('a'):
            if a.text in dividend_keywords:
                get_dividends(urljoin(url, a['href']))


def find_dividends():
    with open('Company URL - Sheet1.csv') as f:
        reader = csv.DictReader(f)
        for row in reader:
            if row['COUNTRY'] == 'USA':
                urls.append(row['URL'])

    for i in range(1, 50):
        print('ON url :: ', urls[i])
        try:
            # page = requests.get(urls[i], headers=headers, timeout=20).text
            driver.get(urls[i])
            page = driver.page_source
        except Exception as err:
            print(err)
            continue
        soup = BeautifulSoup(page, 'lxml')
        print(urls[i])
        print(len(soup.find_all('a')))
        for a in soup.find_all('a', href=True):
            if a.text in investor_relations_keywords:
                go_to_ir(urljoin(urls[i], a['href']))
            elif a.text in dividend_keywords:
                get_dividends(urljoin(urls[i], a['href']))


if __name__ == "__main__":
  #  find_dividends()
  find_news_announcements()

internationalpaper
https://www.internationalpaper.com/performance/news-releases
https://www.internationalpaper.com/newsroom/news-releases
hyster-yale
http://www.hyster-yale.com/News-Room/News-Releases
http://www.hyster-yale.com/news-room/news-releases/default.aspx
http://www.hyster-yale.com/News-Room/News-Releases/default.aspx
1stconstitution
ahbelo
http://www.ahbelo.com/newsroom/
1stconstitution
1stsource
https://www.1stsource.com/about/investor-relations/news-presentations/press-releases/default.aspx
3m
ahbelo
http://www.ahbelo.com/newsroom/
accobrands
acnb
adtran
ads-pipe
amg
https://ir.amg.com/#press-releases
aflac
agcocorp
com
alx-inc
https://www.alx-inc.com/press-release/default.aspx
https://www.alx-inc.com/press-release/default.aspx
https://www.alx-inc.com/press-release/default.aspx
allegiancebank
allete
http://investor.allete.com/press-releases
alliancedata
Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: headless chrome=86.0.4240.198)

alliedmotion
https://w

In [ ]:
!pwd

/content


In [7]:
!zip -r news.zip news

  adding: news/ (stored 0%)
  adding: news/fortive49.html (deflated 89%)
  adding: news/power6.html (deflated 74%)
  adding: news/alx-inc39.html (deflated 78%)
  adding: news/suncoke20.html (deflated 76%)
  adding: news/cimarex19.html (deflated 82%)
  adding: news/olysteel33.html (deflated 83%)
  adding: news/columbiabank10.html (deflated 80%)
  adding: news/columbiabank53.html (deflated 79%)
  adding: news/1stsource28.html (deflated 83%)
  adding: news/columbiabank41.html (deflated 80%)
  adding: news/ssctech13.html (deflated 90%)
  adding: news/firstbankonline3.html (deflated 84%)
  adding: news/pennantpark15.html (deflated 89%)
  adding: news/remax3.html (deflated 74%)
  adding: news/amtd2.html (deflated 85%)
  adding: news/olin2.html (deflated 84%)
  adding: news/jackhenry4.html (deflated 93%)
  adding: news/origin6.html (deflated 90%)
  adding: news/great-ajax1.html (deflated 72%)
  adding: news/suncoke43.html (deflated 76%)
  adding: news/columbiabank50.html (deflated 71%)
  addi

In [8]:
from google.colab import files
import time

files.download('news.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>